## Instalar bibliotecas

In [ ]:
!pip install torch                    # Install the PyTorch library for deep learning.
!pip install pandas                   # Install the Pandas library for data manipulation.
!pip install scikit-learn             # Install scikit-learn for machine learning tasks.
!pip install sentencepiece            # Install SentencePiece for text tokenization.

## Leer archivos de drive

In [1]:
from google.colab import drive

# Montar Google Drive en /content/drive
drive.mount('/content/drive')

# Listar archivos en el directorio raíz de Google Drive
!ls '/content/drive/MyDrive/Dataset/'

Mounted at /content/drive
imagenes_mosaico  test_mosaico.csv  train_mosaico.csv


## Bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torchvision.io import read_image
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader,ConcatDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch import nn
from torch import optim
import datetime
from PIL import Image

## Cargar datasets

In [2]:
# df_train = pd.read_csv('/content/drive/MyDrive/Dataset/train_mosaico.csv')
df_train = pd.read_csv('../Dataset/train.csv')

df_train.head()

,imagen_mosaico,bathrooms,bedrooms,area,zipcode,price
0,285.png,5.0,5,3816,92880,589900
1,348.png,2.0,2,1440,92276,106000
2,441.png,3.0,4,1625,93510,639000
3,422.png,3.0,4,2454,93510,5858000
4,150.png,4.5,4,4038,92677,1795000


## Procesar dataset(imagenes y datos categoricos)



In [8]:
class ProcesarDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform  # Transformación de la imagen
        # self.onehot = onehot        # Codificación one-hot para categorías

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Obtener fila del registro
        tabular = self.df.iloc[idx, :]

        # Precio de la casa
        price = tabular['price']

        # Obtener imagen
        # imgs = read_image(f"/content/drive/MyDrive/Dataset/imagenes_mosaico/{tabular['imagen_mosaico']}").float()
        imgs = read_image(f"../Dataset/imagenes_mosaico/{tabular['imagen_mosaico']}").float()

        # Transformar imagen
        try:
          if self.transform:
            imgs = self.transform(imgs)
        except:
          print("[-]Algo ocurrio con la transformarción de la imagen")

        # Convertir datos tabulares a tensores
        tabular = tabular[["bathrooms", "bedrooms", "area", "zipcode"]]
        tabular = tabular.tolist()
        numeric_features = torch.FloatTensor(tabular)

        # Regresar tupla de valores
        # return image, tabular, y
        return imgs, numeric_features, price


In [9]:
dataset = ProcesarDataset(df_train)

dataset[0][-1]

589900

In [18]:
dataset = ProcesarDataset(
    df = df_train,
    transform = transforms.Compose(
        [
            transforms.Resize(256),
            transforms.RandomCrop((224,224)),
            transforms.Normalize(
                (58.0583, 55.1679, 52.9831),
                (85.9875, 82.3628, 80.8718))
        ]
    )
)


In [19]:
# Especifica la longitud para el conjunto de entrenamiento y validación
train_size = int(0.8 * len(df_train))
val_size = len(df_train) - train_size

# Divide el conjunto de datos
train_data, val_data = torch.utils.data.random_split(dataset, [train_size, val_size])

In [20]:
# Establece el tamaño del lote (batch_size) que se utilizará durante el entrenamiento.
batch_size = 64

# Crea un DataLoader para el conjunto de entrenamiento (train_data).
# - batch_size: Número de muestras en cada lote.
# - shuffle: Mezcla los datos en cada época para introducir variabilidad en el entrenamiento.
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

# Crea un DataLoader para el conjunto de validación (val_data).
# - batch_size: Número de muestras en cada lote.
# - shuffle: Mezcla los datos en cada época para introducir variabilidad en la validación.
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=True)


## Arquitectura

In [31]:
# Definición de una red neuronal utilizando la clase nn.Module de PyTorch.

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        # Capas para procesar características de imágenes
        self.image_features_ = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=2, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(),
            nn.Conv2d(16, 128, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(),
            nn.Conv2d(128, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(),
            nn.Conv2d(256, 128, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(128, 64, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # Capas para procesar características numéricas
        self.numeric_features_ = nn.Sequential(
            nn.Linear(9, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(64, 64*3),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(64*3, 64*3*3),
            nn.ReLU(inplace=True),
        )

        # Capas para combinar características de imágenes y numéricas
        self.combined_features_ = nn.Sequential(
            nn.Linear(64*3*3*2, 64*3*3*2*2),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(64*3*3*2*2, 64*3*3*2),
            nn.ReLU(inplace=True),
            nn.Linear(64*3*3*2, 64),
            nn.Linear(64, 5),
        )

    def forward(self, x, y):
        # Propagación hacia adelante
        x = self.image_features_(x)
        x = x.view(-1, 64*3*3)
        # x = x.view(-1, 20*20*128)
        y = self.numeric_features_(y)
        z = torch.cat((x, y), 1)
        z = self.combined_features_(z)
        return z


## Funciones de optimización y de perdida

In [22]:
# Tipo de dispositivo a usar
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('[+]Se esta usando: {}'.format(device))

# Mandar modelo a dispositivo
model = NeuralNetwork().to(device)

# Crear funciones de optimización y perdida
optimizer=optim.Adam(model.parameters(),1e-3)
loss_fn=nn.CrossEntropyLoss()

[+]Se esta usando: cpu


## Funciones de train y test

In [23]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for imgs,numeric_features, price in dataloader:
            imgs=imgs.to(device)
            numeric_features=numeric_features.to(device)
            price = price.to(device)
            pred = model(imgs,numeric_features)
            test_loss += loss_fn(pred, price).item()
            correct += (pred.argmax(1) == price).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [24]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader):

    print("Número de lotes en train_loader:", len(train_loader))
    print("Número de lotes en val_loader:", len(val_loader))

    total_steps = len(train_loader) * n_epochs  # Total de pasos a realizar
    current_step = 0  # Paso actual

    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, numeric_features, price in train_loader:
            current_step += 1  # Incrementa el paso actual
            imgs = imgs.to(device)
            numeric_features = numeric_features.to(device)
            price = price.to(device)
            output = model(imgs, numeric_features)

            loss = loss_fn(output, price)

            # L2 Regularization
            l2_lambda = 0.001
            l2_norm = sum(p.pow(2).sum() for p in model.parameters())
            loss = loss + l2_lambda * l2_norm

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

            # Calcular el porcentaje de avance y mostrarlo
            percent_complete = (current_step / total_steps) * 100
            print(f'\rEpoch {epoch}, {percent_complete:.2f}% completado. Loss: {loss_train / current_step:.4f}', end='')

        # Al final de cada época, imprime el resumen y evalúa en el conjunto de validación
        if epoch == 1 or epoch % 10 == 0:
            print(f'\n{datetime.datetime.now()} Epoch {epoch}, Training loss {loss_train / len(train_loader)}')
            test_loop(dataloader=val_loader, model=model, loss_fn=loss_fn)
        else:
            # Imprime una nueva línea si no es una época de impresión, para evitar sobrescribir mensajes futuros
            print()



## Train

In [32]:
training_loop(
n_epochs = 100,
optimizer = optimizer,
model = model,
loss_fn = loss_fn,
train_loader = train_dataloader,
val_loader=val_dataloader)

Número de lotes en train_loader: 6
Número de lotes en val_loader: 2


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x4 and 9x64)